In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as dt


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct

# NYC_Restaurant_Inspection_Results_Clean2020



In [2]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///NYC_Restaurant_Inspection_Results_Clean2020.sqlite")
conn = engine.connect()

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
inspector = inspect(engine)
inspector.get_table_names()

['NYCInspect']

In [3]:
Base.classes.keys()

['NYCInspect']

In [4]:
# checking out columns for 'NYCInspect'
columnsN = inspector.get_columns('NYCInspect')
for n in columnsN:
    print(n['name'], n['type'])

id INTEGER
camis INTEGER
dba VARCHAR
bobo VARCHAR
building VARCHAR
street VARCHAR
zipcode VARCHAR
phone VARCHAR
cuisne_description VARCHAR
inspection_date VARCHAR
action VARCHAR
violation_code VARCHAR
violation_description  VARCHAR
critical_flag VARCHAR
score INTEGER
grade VARCHAR
grade_date VARCHAR
record_date VARCHAR
inspection_type VARCHAR
inspection_type_general VARCHAR
latitude FLOAT
longitude FLOAT
community_board VARCHAR
council_district VARCHAR
census_tract VARCHAR
bin VARCHAR
bbl INTEGER
nta VARCHAR
year INTEGER
address VARCHAR


In [5]:
# viewing 'NYCInspect' in DF format
nycDF = pd.read_sql("SELECT * FROM NYCInspect", conn)
nycDF.head()

,id,camis,dba,bobo,building,street,zipcode,phone,cuisne_description,inspection_date,...,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,year,address
0,1,50090691,SALIM KENNEDY FRIED CHICKEN,Queens,16614,HILLSIDE AVE,11432,7183744616,Bangladeshi,2/20/2020,...,40.709572,-73.796195,412,27,46000,4210082,4098180062,QN61,2020,"16614\,HILLSIDE AVE\,11432"
1,2,50095392,HOPE TO SEE YOU,Queens,133-42,39 AVENUE,11354,7188881669,Chinese,1/2/2020,...,40.759099,-73.834139,407,20,87100,4596186,4049737501,QN22,2020,"133-42\,39 AVENUE\,11354"
2,3,50033575,JACQUES TORRES ICE CREAM,Manhattan,89,EAST 42 STREET,10017,2129837353,"Ice Cream\, Gelato\, Yogurt\, Ices",3/9/2020,...,40.752094,-73.977604,105,4,9200,1035381,1012800001,MN19,2020,"89\,EAST 42 STREET\,10017"
3,4,50089342,REGAL CINEMAS ESSEX CROSSING,Manhattan,129,DELANCEY STREET,10002,2124755768,American,3/13/2020,...,40.718327,-73.987518,103,1,1800,1090554,1003527501,MN27,2020,"129\,DELANCEY STREET\,10002"
4,5,50033875,ARIANA'S,Staten Island,60,SOUTH BRIDGE STREET,10309,7189678500,Italian,1/23/2020,...,40.525075,-74.236617,503,51,22600,5000000,5075840117,SI11,2020,"60\,SOUTH BRIDGE STREET\,10309"


In [7]:
groupDF = nycDF.groupby("grade")["camis"].nunique()
groupDF

grade
     3095
A    5561
B     462
C     122
N     449
P      21
Z     199
Name: camis, dtype: int64

In [8]:
# Save references to each table
nyc = Base.classes.NYCInspect

# Create our session (link) from Python to the DB
session = Session(engine)

In [15]:
# nycLat = session.query(nyc.Latitude).\
# filter(nyc.GRADE != '').\
# order_by(nyc.GRADE).all()
# nycbyG

nycLat = session.query(nyc.latitude, nyc.longitude).distinct().filter(nyc.latitude != 0.0).all()

all_latslongs = []
    
for lat, long in nycLat:
    nycLat_dict = {}
    nycLat_dict["lat"] = lat
    nycLat_dict["long"] = long
    all_latslongs.append(nycLat_dict)   
    
all_latslongs


[{'lat': 40.70957152, 'long': -73.79619469},
 {'lat': 40.75909891, 'long': -73.83413912},
 {'lat': 40.75209393, 'long': -73.97760435},
 {'lat': 40.71832661, 'long': -73.98751821},
 {'lat': 40.52507484, 'long': -74.23661714},
 {'lat': 40.86814659, 'long': -73.83143638},
 {'lat': 40.67623155, 'long': -73.92190122},
 {'lat': 40.77734614, 'long': -73.95213342},
 {'lat': 40.79127693, 'long': -73.97240151},
 {'lat': 40.66987092, 'long': -73.90937111},
 {'lat': 40.83235254, 'long': -73.86470087},
 {'lat': 40.69791572, 'long': -73.92673347},
 {'lat': 40.71409444, 'long': -73.94888202},
 {'lat': 40.79546238, 'long': -73.97119709},
 {'lat': 40.75980495, 'long': -73.96993112},
 {'lat': 40.76161637, 'long': -73.96048343},
 {'lat': 40.76074642, 'long': -73.98352505},
 {'lat': 40.75773232, 'long': -73.86354275},
 {'lat': 40.63022448, 'long': -74.01501594},
 {'lat': 40.76366473, 'long': -73.98879842},
 {'lat': 40.72032771, 'long': -73.98918092},
 {'lat': 40.64772845, 'long': -73.97279986},
 {'lat': 4

In [13]:
# # nycLat = session.query(nyc.Latitude).\
# # filter(nyc.GRADE != '').\
# # order_by(nyc.GRADE).all()
# # nycbyG

# nycInfo = session.query(func.count(nyc.DBA)).all()

# nycInfo = session.query(func.count(distinct(nyc.DBA))).all()


nycInfo = session.query(nyc.dba, nyc.bobo, nyc.phone, nyc.cuisne_description, nyc.address).distinct().all()
nycInfo

# all_latslongs = []
    
# for lat, long in nycLat:
#     nycLat_dict = {}
#     nycLat_dict["lat"] = lat
#     nycLat_dict["long"] = long
#     all_latslongs.append(nycLat_dict)   
    
# all_latslongs

# results = session.query(nyc.GRADE).\
# filter(nyc.GRADE != "").\
# filter(nyc.GRADE != "Z").\
# filter(nyc.GRADE != "P").\
# filter(nyc.GRADE != "N").\
# group_by(nyc.CAMIS).\
# order_by(nyc.GRADE).\
# distinct().all()
# results



[('SALIM KENNEDY FRIED CHICKEN',
  'Queens',
  '7183744616',
  'Bangladeshi',
  '16614\\,HILLSIDE AVE\\,11432'),
 ('HOPE TO SEE YOU',
  'Queens',
  '7188881669',
  'Chinese',
  '133-42\\,39 AVENUE\\,11354'),
 ('JACQUES TORRES ICE CREAM',
  'Manhattan',
  '2129837353',
  'Ice Cream\\, Gelato\\, Yogurt\\, Ices',
  '89\\,EAST   42 STREET\\,10017'),
 ('REGAL CINEMAS ESSEX CROSSING',
  'Manhattan',
  '2124755768',
  'American',
  '129\\,DELANCEY STREET\\,10002'),
 ("ARIANA'S",
  'Staten Island',
  '7189678500',
  'Italian',
  '60\\,SOUTH BRIDGE STREET\\,10309'),
 ('POPEYES',
  'Bronx',
  '7183203600',
  'Chicken',
  '360\\,BAYCHESTER AVENUE\\,10475'),
 ('XCELLENT SOUL FOOD',
  'Brooklyn',
  '3476616112',
  'Soul Food',
  '372\\,RALPH AVENUE\\,11233'),
 ('THE DAISY',
  'Manhattan',
  '6469645756',
  'American',
  '1641\\,2 AVENUE\\,10028'),
 ("EDGAR'S CAFE",
  'Manhattan',
  '2125956261',
  'Café/Coffee/Tea',
  '650\\,AMSTERDAM AVENUE\\,10025'),
 ('PITKIN CARIBBEAN BAKERY',
  'Brooklyn',
  '

In [16]:
nycsmall_map = session.query(nyc.dba, nyc.latitude, nyc.longitude).distinct().filter(nyc.latitude != 0.0).all()

small_map = []
    
for dba, lat, long in nycsmall_map:
    nycsmall_map_dict = {}
    nycsmall_map_dict["dba"] = dba
    nycsmall_map_dict["lat"] = lat
    nycsmall_map_dict["long"] = long
    small_map.append(nycsmall_map_dict)   
    
small_map

[{'dba': 'SALIM KENNEDY FRIED CHICKEN',
  'lat': 40.70957152,
  'long': -73.79619469},
 {'dba': 'HOPE TO SEE YOU', 'lat': 40.75909891, 'long': -73.83413912},
 {'dba': 'JACQUES TORRES ICE CREAM', 'lat': 40.75209393, 'long': -73.97760435},
 {'dba': 'REGAL CINEMAS ESSEX CROSSING',
  'lat': 40.71832661,
  'long': -73.98751821},
 {'dba': "ARIANA'S", 'lat': 40.52507484, 'long': -74.23661714},
 {'dba': 'POPEYES', 'lat': 40.86814659, 'long': -73.83143638},
 {'dba': 'XCELLENT SOUL FOOD', 'lat': 40.67623155, 'long': -73.92190122},
 {'dba': 'THE DAISY', 'lat': 40.77734614, 'long': -73.95213342},
 {'dba': "EDGAR'S CAFE", 'lat': 40.79127693, 'long': -73.97240151},
 {'dba': 'PITKIN CARIBBEAN BAKERY', 'lat': 40.66987092, 'long': -73.90937111},
 {'dba': 'NEW NO.1 CHINESE RESTAURANT',
  'lat': 40.83235254,
  'long': -73.86470087},
 {'dba': 'KICHIN', 'lat': 40.69791572, 'long': -73.92673347},
 {'dba': 'ALLIGATOR LOUNGE', 'lat': 40.71409444, 'long': -73.94888202},
 {'dba': 'TEXAS ROTISSERIE & GRILL', 'la

In [18]:
nycgauge = session.query(nyc.dba, nyc.score).distinct().filter(nyc.latitude != 0.0).all()

gauge = []
    
for dba, score in nycgauge:
    gauge_dict = {}
    gauge_dict["dba"] = dba
    gauge_dict["score"] = score
    gauge.append(gauge_dict)   
    
gauge

[{'dba': 'SALIM KENNEDY FRIED CHICKEN', 'score': 20},
 {'dba': 'HOPE TO SEE YOU', 'score': 13},
 {'dba': 'JACQUES TORRES ICE CREAM', 'score': 40},
 {'dba': 'REGAL CINEMAS ESSEX CROSSING', 'score': 2},
 {'dba': "ARIANA'S", 'score': 9},
 {'dba': 'POPEYES', 'score': -1},
 {'dba': 'XCELLENT SOUL FOOD', 'score': 24},
 {'dba': 'THE DAISY', 'score': 63},
 {'dba': "EDGAR'S CAFE", 'score': 12},
 {'dba': 'PITKIN CARIBBEAN BAKERY', 'score': 9},
 {'dba': 'NEW NO.1 CHINESE RESTAURANT', 'score': 19},
 {'dba': 'KICHIN', 'score': 11},
 {'dba': 'ALLIGATOR LOUNGE', 'score': 12},
 {'dba': 'TEXAS ROTISSERIE & GRILL', 'score': 12},
 {'dba': 'LE CAFE COFFEE', 'score': 7},
 {'dba': 'CASABLANCA GRILL', 'score': 27},
 {'dba': "JUNIOR'S RESTAURANT", 'score': 10},
 {'dba': 'ILUSION TAVERN', 'score': 10},
 {'dba': 'TOP OCEAN CHINESE RESTAURANT', 'score': 30},
 {'dba': 'CURRY-YA', 'score': 13},
 {'dba': 'SAUCE PIZZERIA', 'score': 21},
 {'dba': 'SHENANIGANS PUB', 'score': 11},
 {'dba': 'SARKU JAPAN', 'score': 9},
 